# Debug Drill: The Forgotten Context

**Scenario:**
Your support chatbot is giving inconsistent answers.

"I told it my order number at the start, but then it asked me for it again!" users complain.

The bot seems to "forget" information from earlier in the conversation.

**Your Task:**
1. Understand why the transformer forgets early context
2. Simulate context window limitations
3. Implement a solution
4. Write a 3-bullet postmortem

---

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
# Simulate a conversation that exceeds context window

conversation = [
    {"role": "user", "content": "Hi, I need help with order ORD-12345"},
    {"role": "assistant", "content": "I'd be happy to help with order ORD-12345. What's the issue?"},
    {"role": "user", "content": "The item arrived damaged. Can I get a replacement?"},
    {"role": "assistant", "content": "I'm sorry to hear that. Yes, you're eligible for a free replacement. I'll start the process now."},
    # ... many more exchanges about the replacement process ...
    {"role": "user", "content": "Great, thanks! Can you also check if there's a shipping discount on my account?"},
    {"role": "assistant", "content": "Let me check your account for any available discounts."},
    {"role": "user", "content": "Also, I noticed I was charged twice last month. Can you look into that?"},
    {"role": "assistant", "content": "I'll investigate the billing issue. Can you confirm your account email?"},
    {"role": "user", "content": "It's user@example.com. By the way, when will the replacement arrive?"},
    # Bot forgets the order number from the beginning!
    {"role": "assistant", "content": "To check the replacement status, could you please provide your order number?"},  # <-- BUG!
]

print("=== Conversation (Notice the Bug) ===")
for msg in conversation:
    prefix = "User:" if msg['role'] == 'user' else "Bot:"
    print(f"{prefix} {msg['content']}")
    if "could you please provide your order number" in msg['content']:
        print("\n❌ BUG: Bot asked for order number that was given at the start!")

In [ ]:
# ===== COLLEAGUE'S CODE (BUG: No context management) =====

def count_tokens(text):
    """Rough token count (words * 1.3 for English)."""
    return int(len(text.split()) * 1.3)

def format_conversation(messages):
    """Format conversation for LLM."""
    return "\n".join([f"{m['role']}: {m['content']}" for m in messages])

# Simulate a small context window (like an older model)
MAX_CONTEXT_TOKENS = 100  # Very small for demo

def naive_chat(messages, max_tokens=MAX_CONTEXT_TOKENS):
    """Naive approach: just truncate from the beginning."""
    formatted = format_conversation(messages)
    total_tokens = count_tokens(formatted)
    
    # Simple truncation: drop early messages
    while count_tokens(format_conversation(messages)) > max_tokens and len(messages) > 2:
        messages = messages[1:]  # Drop oldest message
    
    return messages

# Simulate what the model sees
truncated = naive_chat(conversation.copy())

print("=== What the Model Actually Sees (After Truncation) ===")
print(f"Original messages: {len(conversation)}")
print(f"After truncation: {len(truncated)}")
print()
for msg in truncated:
    print(f"{msg['role']}: {msg['content'][:50]}...")

print("\n❌ The order number from the first message is GONE!")

---

## Your Investigation

### Step 1: Understand context window limitations

In [ ]:
print("=== Context Window Analysis ===")
print()
print("Transformers have a fixed context window (max input tokens).")
print("When conversation exceeds this, something must be dropped.")
print()
print("Naive approach: Drop oldest messages")
print("  ❌ Loses important early context (order numbers, names)")
print()
print("Better approaches:")
print("  1. Summarize early conversation")
print("  2. Extract and preserve key facts")
print("  3. Use sliding window with summary")
print("  4. Store facts in external memory (database)")

### Step 2: TODO - Implement smart context management

In [ ]:
# TODO: Extract key facts from conversation

# Uncomment and complete:

# import re
# 
# def extract_key_facts(messages):
#     """Extract important facts that should never be forgotten."""
#     facts = {}
#     
#     for msg in messages:
#         content = msg['content']
#         
#         # Extract order numbers
#         order_match = re.search(r'ORD-\d+', content)
#         if order_match:
#             facts['order_id'] = order_match.group()
#         
#         # Extract email
#         email_match = re.search(r'[\w.-]+@[\w.-]+\.\w+', content)
#         if email_match:
#             facts['email'] = email_match.group()
#         
#         # Extract issues mentioned
#         if 'damaged' in content.lower():
#             facts['issue'] = 'damaged item'
#         if 'replacement' in content.lower():
#             facts['resolution'] = 'replacement requested'
#     
#     return facts
# 
# facts = extract_key_facts(conversation)
# print("=== Extracted Key Facts ===")
# for key, value in facts.items():
#     print(f"  {key}: {value}")

In [ ]:
# TODO: Build context with preserved facts

# Uncomment:

# def smart_context(messages, facts, max_tokens=MAX_CONTEXT_TOKENS):
#     """
#     Build context that preserves key facts.
#     
#     Structure:
#     1. System message with key facts
#     2. Recent conversation (as much as fits)
#     """
#     # Create system context with facts
#     facts_summary = "Key facts from this conversation:\n"
#     for key, value in facts.items():
#         facts_summary += f"- {key}: {value}\n"
#     
#     system_msg = {"role": "system", "content": facts_summary}
#     system_tokens = count_tokens(facts_summary)
#     
#     # Fill remaining space with recent messages
#     remaining_tokens = max_tokens - system_tokens
#     recent_messages = []
#     
#     for msg in reversed(messages):
#         msg_tokens = count_tokens(msg['content'])
#         if msg_tokens <= remaining_tokens:
#             recent_messages.insert(0, msg)
#             remaining_tokens -= msg_tokens
#         else:
#             break
#     
#     return [system_msg] + recent_messages
# 
# smart = smart_context(conversation, facts)
# 
# print("=== Smart Context (Facts Preserved) ===")
# for msg in smart:
#     print(f"{msg['role']}: {msg['content'][:60]}..." if len(msg['content']) > 60 else f"{msg['role']}: {msg['content']}")

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

# Uncomment:

# assert 'order_id' in facts, "Should extract order ID"
# assert facts['order_id'] == 'ORD-12345', f"Order ID should be ORD-12345, got {facts.get('order_id')}"
# 
# # Check that smart context includes the order
# context_text = ' '.join([m['content'] for m in smart])
# assert 'ORD-12345' in context_text, "Smart context should preserve order ID"
# 
# print("✓ Context management fixed!")
# print(f"✓ Order ID preserved: {facts['order_id']}")
# print(f"✓ Context includes key facts")

### Step 3: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Forgotten Context

### What happened:
- (Your answer: What user complaint indicated the bot was forgetting context?)

### Root cause:
- (Your answer: Why do transformers forget early conversation?)

### How to prevent:
- (Your answer: What strategies preserve important context?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **Transformers have fixed context windows.** They can only see a limited amount of text.

2. **Naive truncation loses important early context.** Order numbers, names, issues get dropped.

3. **Extract and preserve key facts.** Use regex or NER to identify important entities.

4. **Add facts to system prompt.** Ensures they're always in context.

---

## Context Management Strategies

| Strategy | Pros | Cons |
|----------|------|------|
| Naive truncation | Simple | Loses early context |
| Summarization | Preserves gist | Loses specifics |
| Fact extraction | Keeps key info | Requires parsing |
| External memory | Unlimited history | Added complexity |